In [14]:
import log_reg
from imp import reload

import numpy as np
import pandas as pd

import evaluation
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
Y = 10
vocab_min = 3

In [3]:
reload(log_reg)
notefile = '../mimicdata/notes_' + str(Y) + '_train_ind.csv'
X_tr = log_reg.construct_csr_matrix(notefile)

Processing...............................................................

In [4]:
X_tr.shape

(15492, 40000)

In [6]:
reload(log_reg)
labelfile = '../mimicdata/labels_10_train.csv'
yy_tr = log_reg.construct_label_lists(labelfile, Y, X_tr.shape[0])

In [7]:
yy_tr

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ..., 
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [8]:
devfile = '../mimicdata/notes_' + str(Y) + '_dev_ind.csv'
X_dv = log_reg.construct_csr_matrix(devfile)

Processing.................................

In [10]:
X_dv.shape

(7746, 40000)

Still have to de-dupe label set for some strange reason. Maybe I'll just re-export from SQL after doing a DISTINCT

^^ actually, my construct_label_lists takes care of this

In [11]:
labelfile_dev = '../mimicdata/labels_10_dev.csv'
yy_dev = log_reg.construct_label_lists(labelfile_dev, Y, X_dv.shape[0])

In [16]:
import warnings
warnings.filterwarnings("ignore")

## LogReg

In [28]:
#optimize hyperparams of LR
reload(evaluation)
iters = [3, 5, 7, 10]
Cs = [1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]
clfs = []
for mx_iter in iters:
    for C in Cs:
        print("C = " + str(C) + ", iters = " + str(mx_iter) + ": ")
        reg = OneVsRestClassifier(LogisticRegression(C=C,solver='sag', max_iter=mx_iter))
        reg.fit(X_tr, yy_tr)
        yhat = reg.predict(X_dv)
        acc,precision,recall,f1 = evaluation.all_metrics(yhat, yy_dev)
        print("acc,precision,recall,f1")
        print(acc,precision,recall,f1)
        clfs.append(reg)

C = 0.01, iters = 3: 
acc,precision,recall,f1
0.212631814065 0.505043770671 0.227132795626 0.284060308828
C = 0.1, iters = 3: 
acc,precision,recall,f1
0.212949182169 0.507765605597 0.22512356608 0.283318183667
C = 1.0, iters = 3: 
acc,precision,recall,f1
0.224853175628 0.529577631466 0.241988952824 0.299486361531
C = 10.0, iters = 3: 
acc,precision,recall,f1
0.230908784462 0.543378610158 0.247985340104 0.307215423143
C = 100.0, iters = 3: 
acc,precision,recall,f1
0.225062756405 0.534480776561 0.235581651974 0.299071554145
C = 1000.0, iters = 3: 
acc,precision,recall,f1
0.221028108722 0.525535760392 0.235174273255 0.294091905169
C = 0.01, iters = 5: 
acc,precision,recall,f1
0.225129867336 0.531988553232 0.238978797044 0.299352091664
C = 0.1, iters = 5: 
acc,precision,recall,f1
0.224048715364 0.531778818766 0.235495278669 0.297321999801
C = 1.0, iters = 5: 
acc,precision,recall,f1
0.224984938463 0.533249214136 0.23764907848 0.298884993424
C = 10.0, iters = 5: 
acc,precision,recall,f1
0.2

## SVM

In [32]:
from sklearn.svm import SVC
iters = [3, 5, 7, 10]
Cs = [1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]
svms = []
for mx_iter in iters:
    for C in Cs:
        print("C = " + str(C) + ", iters = " + str(mx_iter) + ": ")
        svm = OneVsRestClassifier(SVC(C=C, max_iter=mx_iter))
        svm.fit(X_tr, yy_tr)
        yhat = svm.predict(X_dv)
        acc,precision,recall,f1 = evaluation.all_metrics(yhat, yy_dev)
        print("acc,precision,recall,f1")
        print(acc,precision,recall,f1)
        svms.append(svm)

C = 0.01, iters = 3: 
acc,precision,recall,f1
0.20753594277 0.244468471182 0.609138562043 0.311823809752
C = 0.1, iters = 3: 
acc,precision,recall,f1
0.20753594277 0.244468471182 0.609138562043 0.311823809752
C = 1.0, iters = 3: 
acc,precision,recall,f1
0.20753594277 0.244468471182 0.609138562043 0.311823809752
C = 10.0, iters = 3: 
acc,precision,recall,f1
0.206748285854 0.25461677097 0.589105693056 0.310577218286
C = 100.0, iters = 3: 
acc,precision,recall,f1
0.0880359140816 0.112800923364 0.242436137033 0.13221510305
C = 1000.0, iters = 3: 
acc,precision,recall,f1
0.122095172521 0.189635203423 0.24662826487 0.181343287494
C = 0.01, iters = 5: 
acc,precision,recall,f1
0.206559658031 0.253922864438 0.569262066648 0.308291373023
C = 0.1, iters = 5: 
acc,precision,recall,f1
0.206559658031 0.253922864438 0.569262066648 0.308291373023
C = 1.0, iters = 5: 
acc,precision,recall,f1
0.206559658031 0.253922864438 0.569262066648 0.308291373023
C = 10.0, iters = 5: 
acc,precision,recall,f1
0.2034

KeyboardInterrupt: 

In [18]:
pcts = sorted([np.array(yy_dev[i]).mean() for i in range(Y)])
pcts
#these are the % of instances containing this label for each of the top 10 labels

[0.18409501678285567,
 0.18874257681383941,
 0.2101729925122644,
 0.24141492383165505,
 0.26452362509682414,
 0.2747224373870385,
 0.3187451587916344,
 0.31900335657113349,
 0.32726568551510454,
 0.55499612703330747]

In [20]:
accs = [(yhat_tr == yy[i]).mean() for i in range(Y)]
accs

[0.77956364575264647,
 0.65085205267234703,
 0.45339530080041313,
 0.63665117479989675,
 0.77285050348566997,
 0.88187451587916343,
 0.81493674154402274,
 0.67867286341337463,
 0.86586625355021951,
 0.78563129357087524]

In [22]:
1 - np.array(pcts).mean()
np.array(accs).mean()

0.73202943454686298

In [40]:
yhat_tr = svms[0].predict(X_tr)

In [42]:
evaluation.all_metrics(yhat_tr, yy_tr)

(0.19150234734568436,
 0.21784384874528107,
 0.61208296447936339,
 0.28794924481589851)

In [44]:
yhat_tr_lr = clfs[0].predict(X_tr)

In [46]:
evaluation.all_metrics(yhat_tr_lr, yy_tr)

(0.23745530207091384,
 0.54204274402763963,
 0.23924967417899629,
 0.30920301834321851)

In [ ]:
noreg = OneVsRestClassifier(LogisticRegression(C=np.inf, max_iter=3, solver='sag'))
noreg.fit(X_tr, yy_tr)

In [ ]:
yhat_tr_noreg = noreg.predict(X_tr)
evaluation.all_metrics(yhat_tr_noreg, yy_tr)